In [2]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE
using FUSE.IMAS
using Plots; gr();
using LibGEOS
using Interpolations
using Contour

  Activating environment at `~/.julia/dev/FUSE/Project.toml`
┌ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]
└ @ Base loading.jl:1342


In [3]:


#= ==== =#
#  init  #
#= ==== =#

"""
    init(build::IMAS.radial_build; layers...)

Initialize radial_build IDS based on center stack layers (thicknesses)

NOTE: layer[:].type and layer[:].material follows from naming of layers
*   0 ...gap... : vacuum
*   1 OH: ohmic coil
*   2 TF: toroidal field coil
*   3 shield...: neutron shield
*   4 blanket...: neutron blanket
*   5 wall....: 
*  -1 ...vessel...: 

layer[:].hfs is set depending on if "hfs" or "lfs" appear in the name

layer[:].identifier is created as a hash of then name removing "hfs" or "lfs"

NOTE layer[:].shape integer index corresponds to the following shapes
*   1 : Priceton_D (shape_parameters = [])
*   2 : Rectangle  (shape_parameters = [height])
*   3 : tripple_arc (shape_parameters = [h, small_radius, mid_radius, small_coverage, mid_coverage])
"""
function init(build::IMAS.radial_build; layers...)
    # assign layers
    resize!(build.layer, length(layers))
    for (k, (layer_name, layer_thickness)) in enumerate(layers)
        build.layer[k].thickness = layer_thickness
        build.layer[k].name = replace(String(layer_name), "_" => " ")
        if occursin("gap", lowercase(build.layer[k].name))
            build.layer[k].type = 0
            build.layer[k].material = "vacuum"
        elseif uppercase(build.layer[k].name) == "OH"
            build.layer[k].type = 1
        elseif occursin("TF", uppercase(build.layer[k].name))
            build.layer[k].type = 2
        elseif occursin("shield", lowercase(build.layer[k].name))
            build.layer[k].type = 3
        elseif occursin("blanket", lowercase(build.layer[k].name))
            build.layer[k].type = 4
        elseif occursin("wall", lowercase(build.layer[k].name))
            build.layer[k].type = 5
        end
        if occursin("hfs", lowercase(build.layer[k].name))
            build.layer[k].hfs = 1
        elseif occursin("lfs", lowercase(build.layer[k].name))
            build.layer[k].hfs = -1
        else
            build.layer[k].hfs = 0
        end
        if occursin("vessel", lowercase(build.layer[k].name))
            build.layer[k].type = -1
            build.layer[k].material = "vacuum"
        end
        build.layer[k].identifier = UInt(hash(replace(replace(lowercase(build.layer[k].name), "hfs" => ""), "lfs" => "")))
    end
    if build.layer[end].material != "vacuum"
        error("radial_build last material must be `vacuum`")
    end

    return build
end

"""
    init(build::IMAS.radial_build, eqt::IMAS.equilibrium__time_slice; is_nuclear_facility=true)

Simple initialization of radial_build IDS based on equilibrium time_slice
"""
function init(build::IMAS.radial_build, eqt::IMAS.equilibrium__time_slice; is_nuclear_facility=true, conformal_wall=true)
    rmin = eqt.boundary.geometric_axis.r - eqt.boundary.minor_radius
    rmax = eqt.boundary.geometric_axis.r + eqt.boundary.minor_radius

    if is_nuclear_facility
        n_hfs_layers = 6
        gap = (rmax - rmin) / 20.0 # plasma-wall gap
        rmin -= gap
        rmax += gap
        dr = rmin / n_hfs_layers
        init(build,
            gap_OH=dr * 2.0,
            OH=dr,
            hfs_TF=dr,
            gap_hfs_TF_shield=0.0,
            hfs_shield=dr / 2.0,
            hfs_blanket=dr,
            hfs_wall=dr / 2.0,
            vacuum_vessel=rmax - rmin,
            lfs_wall=dr / 2.0,
            lfs_blanket=dr * 2,
            lfs_shield=dr / 2.0,
            gap_lfs_TF_shield=dr * 5,
            lfs_TF=dr,
            gap_cryostat=5 * dr)

    else
        n_hfs_layers = 4.5
        gap = (rmax - rmin) / 20.0 # plasma-wall gap
        rmin -= gap
        rmax += gap
        dr = rmin / n_hfs_layers
        init(build,
            gap_OH=dr * 2.0,
            OH=dr,
            hfs_TF=dr,
            gap_hfs_TF_wall=0.0,
            hfs_wall=dr / 2.0,
            vacuum_vessel=rmax - rmin,
            lfs_wall=dr / 2.0,
            gap_lfs_TF_wall=dr * 3,
            lfs_TF=dr,
            gap_cryostat=2 * dr)
    end
    # TF shape
    build.layer[3].shape=3
    build.layer[3].shape_parameters= [100.0, 10.0, 30.0, 80.0, 20.0]
    build.tf.coils_n = 16
    radial_build_cx(build, eqt, conformal_wall)

    return build
end

function xy_polygon(x, y)
    if x[1] ≈ x[end]
        x[end] = x[1]
        y[end] = y[1]
    elseif x[1] != x[end]
        push!(x, x[1])
        push!(y, y[1])
    end
    coords = [collect(map(collect, zip(x, y)))]
    return LibGEOS.Polygon(coords)
end

function miller(R0, epsilon, kappa, delta, n)
    θ = range(0, 2pi, length=n)
    δ₀ = asin(delta)
    x = R0 * (1 .+ epsilon .* cos.(θ .+ δ₀ * sin.(θ)))
    y = R0 * (epsilon * kappa * sin.(θ))
    return [x, y]
end

function wall_miller_conformal(build, layer_type, elongation, triangularity)
    if layer_type == -1
        Rstart = IMAS.get_radial_build(build, type=layer_type).start_radius
        Rend = IMAS.get_radial_build(build, type=layer_type).end_radius
        line = miller((Rend + Rstart) / 2.0, (Rend - Rstart) / (Rend + Rstart), elongation, triangularity, 100)        
        return line, line
    else
        Rstart_lfs = IMAS.get_radial_build(build, type=layer_type, hfs=-1).start_radius
        Rend_lfs = IMAS.get_radial_build(build, type=layer_type, hfs=-1).end_radius
        Rstart_hfs = IMAS.get_radial_build(build, type=layer_type, hfs=1).start_radius
        Rend_hfs = IMAS.get_radial_build(build, type=layer_type, hfs=1).end_radius
        inner_line = miller((Rstart_lfs + Rend_hfs) / 2.0, (Rstart_lfs - Rend_hfs) / (Rstart_lfs + Rend_hfs), elongation, triangularity, 100)
        outer_line = miller((Rend_lfs + Rstart_hfs) / 2.0, (Rend_lfs - Rstart_hfs) / (Rend_lfs + Rstart_hfs), elongation, triangularity, 100)
        return inner_line, outer_line
    end
end

function wall_plug(build::IMAS.radial_build, type= :wall_plug)
    L = 0
    R = IMAS.get_radial_build(build, type=1).start_radius
    U = maximum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z)
    D = minimum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z)
    return [L,R,R,L,L], [D,D,U,U,D]
end

function wall_oh(build::IMAS.radial_build)
    L = IMAS.get_radial_build(build, type=1).start_radius
    R = IMAS.get_radial_build(build, type=1).end_radius
    U = maximum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z)
    D = minimum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z)
    return [L,R,R,L,L], [D,D,U,U,D]
end

function wall_cryostat(build::IMAS.radial_build)
    L = 0
    R = build.layer[end].end_radius
    U = maximum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z) + build.layer[end].thickness
    D = minimum(IMAS.get_radial_build(build, type=2, hfs=1).outline.z) - build.layer[end].thickness
    return [L,R,R,L,L], [D,D,U,U,D]
end

"""
    radial_build_cx(build::IMAS.radial_build, eqt::IMAS.equilibrium__time_slice, conformal_wall::Bool=false)
Translates 1D radial build to 2D cross-sections
"""

function radial_build_cx(build::IMAS.radial_build, eqt::IMAS.equilibrium__time_slice, conformal_wall=false)
    # we make the lfs wall to be conformal to miller
    n = floor(Int,length(eqt.profiles_1d.elongation) * 0.95)
    inner_wall_line, outer_wall_line = wall_miller_conformal(build, 5, eqt.profiles_1d.elongation[n], (eqt.profiles_1d.triangularity_upper[n] + eqt.profiles_1d.triangularity_lower[n]) / 2.0) # wall
    outer_wall_line[2] = outer_wall_line[2] .* 1.2
    outer_wall_poly = xy_polygon(outer_wall_line...)
    inner_wall_poly = LibGEOS.buffer(outer_wall_poly, -IMAS.get_radial_build(build, type=5, hfs=1).thickness)

    if ! conformal_wall
        vessel_poly = LibGEOS.buffer(outer_wall_poly, -IMAS.get_radial_build(build, type=5, hfs=1).thickness)
    else
        r = range(eqt.profiles_2d[1].grid.dim1[1], eqt.profiles_2d[1].grid.dim1[end], length=length(eqt.profiles_2d[1].grid.dim1))
        z = range(eqt.profiles_2d[1].grid.dim2[1], eqt.profiles_2d[1].grid.dim2[end], length=length(eqt.profiles_2d[1].grid.dim2))
        PSI_interpolant = Interpolations.CubicSplineInterpolation((r, z), eqt.profiles_2d[1].psi)

        # Inner/lfs radii of the vacuum vessel
        R_hfs_vessel = IMAS.get_radial_build(build, type=-1).start_radius
        R_lfs_vessel = IMAS.get_radial_build(build, type=-1).end_radius
        psi_vessel_trace = PSI_interpolant(R_hfs_vessel, 0)

        # Trace contours of psi and use it as the shape of the vacuum vessel.
        cl = Contour.contour(r, z, eqt.profiles_2d[1].psi, psi_vessel_trace)
        traces = []
        for line in Contour.lines(cl)
            pr, pz = Contour.coordinates(line)
            distance_R_hfs = sqrt.((pr .- R_hfs_vessel).^2 + (pz .- 0.0).^2)
            distance_R_lfs = sqrt.((pr .- R_lfs_vessel).^2 + (pz .- 0.0).^2)
            trace = Dict()
            trace[:pr] = pr
            trace[:pz] = pz
            trace[:d_hfs] = minimum(distance_R_hfs)
            trace[:d_lfs] = minimum(distance_R_lfs)
            trace[:R_hfs] = pr[argmin(distance_R_hfs)]
            trace[:R_lfs] = pr[argmin(distance_R_lfs)]
            trace[:contains] = (sign(pz[1]) == sign(pz[end])) && (PolygonOps.inpolygon((eqt.global_quantities.magnetic_axis.r, eqt.global_quantities.magnetic_axis.z), StaticArrays.SVector.(vcat(pr, pr[1]), vcat(pz, pz[1]))) == 1)
            push!(traces, trace)
        end

        vessel_line = []
        if ! any([trace[:contains] for trace in traces])
            trace_hfs = traces[argmin([trace[:d_hfs] for trace in traces])]
            trace_lfs = traces[argmin([trace[:d_lfs] for trace in traces])]
            hfs_vessel_line = [trace_hfs[:pr],trace_hfs[:pz]]
            lfs_vessel_line = [trace_lfs[:pr],trace_lfs[:pz]]
            if sign(hfs_vessel_line[2][1]) != sign(lfs_vessel_line[2][1])
                vessel_line = [vcat(hfs_vessel_line[1], lfs_vessel_line[1]),
                               vcat(hfs_vessel_line[2][1] * 2,hfs_vessel_line[2][2:end - 1],hfs_vessel_line[2][end] * 2,
                                    lfs_vessel_line[2][1] * 2,lfs_vessel_line[2][2:end - 1],lfs_vessel_line[2][end] * 2)]
            else
                vessel_line = [vcat(hfs_vessel_line[1], reverse(lfs_vessel_line[1])),
                                            vcat(hfs_vessel_line[2][1] * 2,hfs_vessel_line[2][2:end - 1],hfs_vessel_line[2][end] * 2,
                                    reverse(vcat(lfs_vessel_line[2][1] * 2, lfs_vessel_line[2][2:end - 1], lfs_vessel_line[2][end] * 2)))]
            end
            trace = Dict()
            trace[:pr] = vessel_line[1]
            trace[:pz] = vessel_line[2]
            trace[:R_hfs] = trace_hfs[:R_hfs]
            trace[:R_lfs] = trace_lfs[:R_lfs]
            trace[:contains] = true
            push!(traces, trace)
        end
        vessel_line = []
        for trace in traces
            if ! trace[:contains]
                continue
            end
            scale = (R_lfs_vessel .- R_hfs_vessel) ./ (trace[:R_lfs] .- trace[:R_hfs])
            fact = exp.(-(trace[:pz] ./ maximum(abs.(trace[:pz])) .* eqt.boundary.elongation).^2) * (scale - 1) .+ 1
            push!(vessel_line, (trace[:pr] .- (trace[:R_hfs] .+ trace[:R_lfs]) ./ 2) .* fact .+ (R_hfs_vessel .+ R_lfs_vessel) ./ 2)
            push!(vessel_line, trace[:pz] .* fact)
        end
        vessel_poly = xy_polygon(vessel_line...)
        
        # cut the top/bottom part of the vessel with the inner_wall_line
        vessel_poly = LibGEOS.intersection(vessel_poly, inner_wall_poly)

        # make the divertor domes in the vessel
        δψ = 0.05
        cl = Contour.contour(r, z, eqt.profiles_2d[1].psi, eqt.profiles_1d.psi[end] * (1 - δψ) + eqt.profiles_1d.psi[1] * δψ)
        for line in Contour.lines(cl)
            pr, pz = Contour.coordinates(line)
            if pr[1] != pr[end]
                pz[1] = pz[1] * 2
                pz[end] = pz[end] * 2
                vessel_poly = LibGEOS.difference(vessel_poly, xy_polygon(pr, pz))
            end
        end

    end

    # vacuum vessel
    IMAS.get_radial_build(build, type=-1).outline.r = [v[1] for v in LibGEOS.coordinates(vessel_poly)[1]]
    IMAS.get_radial_build(build, type=-1).outline.z = [v[2] for v in LibGEOS.coordinates(vessel_poly)[1]]

    # wall
    IMAS.get_radial_build(build, type=5, hfs=1).outline.r = [v[1] for v in LibGEOS.coordinates(outer_wall_poly)[1]]
    IMAS.get_radial_build(build, type=5, hfs=1).outline.z = [v[2] for v in LibGEOS.coordinates(outer_wall_poly)[1]]

    # all layers between wall and OH
    valid = false
    for (k, layer) in reverse(collect(enumerate(build.layer)))
        # stop once you see the OH
        if layer.type == 1
            valid = false
            break
        end
        if valid
            outer_layer = build.layer[k + 1]
            hfs_thickness = layer.thickness
            lfs_thickness = IMAS.get_radial_build(build, identifier=layer.identifier, hfs=-1).thickness
            poly = LibGEOS.buffer(xy_polygon(outer_layer.outline.r, outer_layer.outline.z), (hfs_thickness + lfs_thickness) / 2.0)
            build.layer[k].outline.r = [v[1] .+ (lfs_thickness .- hfs_thickness) / 2.0 for v in LibGEOS.coordinates(poly)[1]]
            build.layer[k].outline.z = [v[2] for v in LibGEOS.coordinates(poly)[1]]
        end
        # valid starting from the wall
        if (layer.type == 5) && (layer.hfs == 1)
            valid = true
        end
    end

    # if it's a nuclear facility we overwrite TF outer and inner outlines with princeton D
    # for now we do this only if there is a blanket because without it it is likely that the TF and the wall will encroach
    if IMAS.get_radial_build(build, type=4, hfs=1, raise_error_on_missing=false) !== nothing
        layer = IMAS.get_radial_build(build, type=2, hfs=1)
        end_radius = IMAS.get_radial_build(build, identifier=layer.identifier, hfs=-1).start_radius
        start_radius = layer.end_radius
        if layer.shape  == 1
            R_TF, Z_TF = princeton_D(start_radius, end_radius)
        elseif layer.shape == 2
            R_TF, Z_TF = rectangle_shape(start_radius, end_radius, layer.shape_parameters...)
        elseif layer.shape == 3
            R_TF, Z_TF = tripple_arc(start_radius, end_radius, layer.shape_parameters...)

        else
            error("layer.shape $(layer.shape) doesn't exist see: \n            *   1 : Priceton_D (shape_parameters = []) \n
            *   2 : Rectangle  (shape_parameters = [height]) \n
            *   3 : tripple_arc (shape_parameters = [h, small_radius, mid_radius, small_coverage, mid_coverage])")
        end
        poly = LibGEOS.buffer(xy_polygon(R_TF, Z_TF), layer.thickness)
        layer.outline.r = [v[1] for v in LibGEOS.coordinates(poly)[1]]
        layer.outline.z = [v[2] for v in LibGEOS.coordinates(poly)[1]]
        layer = build.layer[IMAS.get_radial_build(build, type=2, hfs=-1, return_index=true) + 1]
        layer.outline.r = R_TF
        layer.outline.z = Z_TF
    end

    # set the toroidal thickness of the TF coils based on the innermost radius and the number of coils
    build.tf.thickness = 2 * π * IMAS.get_radial_build(build, type=2, hfs=1).start_radius / build.tf.coils_n

    # plug
    build.layer[1].outline.r, build.layer[1].outline.z = wall_plug(build)

    # oh
    build.layer[2].outline.r, build.layer[2].outline.z = wall_oh(build)

    # cryostat
    build.layer[end].outline.r, build.layer[end].outline.z = wall_cryostat(build)
    return build
end

radial_build_cx (generic function with 2 methods)